In [1]:
import torch
from sklearn.cluster import KMeans
from fastai.vision.all import *
from fastai.learner import load_learner
from collections import defaultdict
import pandas as pd
from tqdm import tqdm

set_seed(42)

# Function to get the full image path from frames_df
def get_image(r, dataroot_path): 
    return str(dataroot_path / 'frames' / (r['filename'] + '.jpg'))

# Extract the label from the DataFrame row
def get_label(r): 
    return r['task_label']

# Extract case ID from the filename
def extract_case_from_filename(filename): 
    return filename.split('_')[0]  # Assuming filenames are like 'case01_frame123.jpg'

# Extract features using the Fastai learner
def extract_features_using_learner(batch, learn, device):
    dl = learn.dls.test_dl(batch, device=device)
    with torch.no_grad():
        preds = learn.get_preds(dl=dl, with_decoded=False)[0]
    return preds.cpu().numpy()

# Select diverse frames from a set of frame paths
def select_diverse_frames(frame_paths, learn, device, n_select=5, n_clusters=5):
    features = extract_features_using_learner(frame_paths, learn, device)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(features)
    
    selected_frames = []
    for i in range(n_clusters):
        cluster_indices = np.where(kmeans.labels_ == i)[0]
        selected_frame = frame_paths[cluster_indices[0]]
        selected_frames.append(selected_frame)
    
    return selected_frames

# Process all frames, group by case, and select diverse frames
def process_frames(frames_df, dataroot_path, learn, device, n_select=5, n_clusters=5):
    all_selected_frames = []
    
    case_ids = frames_df['case_folder'].unique()
    print(f"Found {len(case_ids)} cases in the dataset.")
    
    for case_id in tqdm(case_ids, desc="Processing cases"):
        case_frames = frames_df[frames_df['case_folder'] == case_id]
        frame_paths = [get_image(row, dataroot_path) for _, row in case_frames.iterrows()]
        
        if len(frame_paths) < n_clusters:
            print(f"Skipping case {case_id} as it has fewer frames than the number of clusters.")
            continue
        
        print(f"Processing case: {case_id} with {len(frame_paths)} frames.")
        selected_frames = select_diverse_frames(frame_paths, learn, device, n_select, n_clusters)
        all_selected_frames.extend(selected_frames)
    
    return all_selected_frames

# Load the model and learner
model_path = Path('./models/baseline/vit_tiny.pkl')  # Adjust this path to your actual model location
learn = load_learner(model_path)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    learn = learn.to_fp16()

# Load the frames dataframe
frames_csv_path = '/home/exsdatalab/data/surgvu24/frames_info.csv'  # Adjust the path if necessary
frames_df = pd.read_csv(frames_csv_path)

# Define the dataroot path
dataroot_path = Path('/home/exsdatalab/data/surgvu24')

# Call the function to process frames
selected_frames = process_frames(frames_df, dataroot_path, learn, device, n_select=5, n_clusters=5)

In [4]:
# Output the selected frames
for frame in selected_frames:
    print(frame)

/home/exsdatalab/data/surgvu24/frames/case_000_video_part_001_t1773.700000_f000427.jpg
/home/exsdatalab/data/surgvu24/frames/case_000_video_part_001_t461.916667_f000111.jpg
/home/exsdatalab/data/surgvu24/frames/case_000_video_part_001_t8.333333_f000002.jpg
/home/exsdatalab/data/surgvu24/frames/case_000_video_part_001_t25.000000_f000006.jpg
/home/exsdatalab/data/surgvu24/frames/case_000_video_part_001_t128.583333_f000031.jpg
/home/exsdatalab/data/surgvu24/frames/case_001_video_part_001_t2316.633333_f000556.jpg
/home/exsdatalab/data/surgvu24/frames/case_001_video_part_001_t6306.550000_f001515.jpg
/home/exsdatalab/data/surgvu24/frames/case_001_video_part_001_t45.833333_f000011.jpg
/home/exsdatalab/data/surgvu24/frames/case_001_video_part_001_t4037.466667_f000969.jpg
/home/exsdatalab/data/surgvu24/frames/case_001_video_part_001_t516.666667_f000124.jpg
/home/exsdatalab/data/surgvu24/frames/case_002_video_part_001_t125.000000_f000030.jpg
/home/exsdatalab/data/surgvu24/frames/case_002_video_p

In [5]:
import os
import shutil
from pathlib import Path

def copy_selected_frames(selected_frames, dataroot_path):
    # Create the destination directory
    frames_seg_folder = dataroot_path / 'frames-seg'
    frames_seg_folder.mkdir(parents=True, exist_ok=True)
    
    # Copy each selected frame to the destination directory
    for frame_path in selected_frames:
        # Get the filename from the frame path
        filename = Path(frame_path).name
        
        # Define the destination path
        dest_path = frames_seg_folder / filename
        
        # Copy the file
        shutil.copy2(frame_path, dest_path)
    
    print(f"Copied {len(selected_frames)} frames to {frames_seg_folder}")

# Example usage:
# Assuming selected_frames is the list of selected frame paths
dataroot_path = Path('/home/exsdatalab/data/surgvu24')
copy_selected_frames(selected_frames, dataroot_path)

Copied 775 frames to /home/exsdatalab/data/surgvu24/frames-seg


In [3]:
def create_prodigy_jsonl(frames_folder, output_jsonl):
    # Get all image files from the frames-seg folder
    image_files = get_image_files(frames_folder)
    
    # Create the JSONL file
    with open(output_jsonl, 'w') as f:
        for img_file in image_files:
            # Create a dictionary for each image
            record = {
                "image": str(img_file.resolve()),  # Full path to the image
                "meta": {
                    "file_name": img_file.name
                }
            }
            # Write each dictionary as a JSON object in the JSONL file
            f.write(json.dumps(record) + '\n')
    
    print(f"JSONL file created with {len(image_files)} records at {output_jsonl}")

In [6]:
# Example usage:
frames_seg_folder = Path('/home/exsdatalab/data/surgvu24/segment/frames')
output_jsonl_path = Path('/home/exsdatalab/data/surgvu24/segment/images.jsonl')
create_prodigy_jsonl(frames_seg_folder, output_jsonl_path)

JSONL file created with 775 records at /home/exsdatalab/data/surgvu24/segment/images.jsonl


In [4]:
# Example usage:
frames_seg_folder = Path('/home/exsdatalab/mlworks/surgvu/submission/frames/vid_1_short')
output_jsonl_path = Path('/home/exsdatalab/data/surgvu24/segment/images2.jsonl')
create_prodigy_jsonl(frames_seg_folder, output_jsonl_path)

JSONL file created with 60 records at /home/exsdatalab/data/surgvu24/segment/images2.jsonl
